In [1]:
import sys
!{sys.executable} -m pip install lightgbm


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score,\
                            accuracy_score, balanced_accuracy_score,classification_report,\
                            ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K
from sklearn.utils import shuffle

np.random.seed(1635848)

In [3]:
class cGAN():
    
    def __init__(self,latent_dim=32, out_shape=14):
        
        self.latent_dim = latent_dim
        self.out_shape = out_shape 
        self.num_classes = 2
        # using Adam as our optimizer
        optimizer = Adam(0.0002, 0.5)
        
        # building the discriminator
        self.discriminator = self.discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # building the generator
        self.generator = self.generator()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        gen_samples = self.generator([noise, label])
        
        # we don't train discriminator when training generator
        self.discriminator.trainable = False
        valid = self.discriminator([gen_samples, label])

        # combining both models
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
                              optimizer=optimizer,
                             metrics=['accuracy'])


    
    def generator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(128, input_dim=self.latent_dim))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(self.out_shape, activation='tanh'))

        noise = Input(shape=(self.latent_dim))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        gen_sample = model(model_input)

        return Model([noise, label], gen_sample, name="Generator")
    

    def discriminator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(512, input_dim=self.out_shape, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(256, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        
        model.add(Dense(128, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        
        model.add(Dense(1, activation='sigmoid'))
        
        gen_sample = Input(shape=(self.out_shape,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.out_shape)(label))

        model_input = multiply([gen_sample, label_embedding])
        validity = model(model_input)

        return Model(inputs=[gen_sample, label], outputs=validity, name="Discriminator")
    

    def train(self, X_train, y_train, pos_index, neg_index, epochs, sampling=False, batch_size=32, sample_interval=100, plot=True): 
        
        # though not recommended, defining losses as global helps as in analysing our cgan out of the class
        global G_losses
        global D_losses
        
        G_losses = []
        D_losses = []
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            
            # if sampling==True --> train discriminator with 8 sample from postivite class and rest with negative class
            if sampling:
                idx1 = np.random.choice(pos_index, 8)
                idx0 = np.random.choice(neg_index, batch_size-8)
                idx = np.concatenate((idx1, idx0))
            # if sampling!=True --> train discriminator using random instances in batches of 32
            else:
                idx = np.random.choice(len(y_train), batch_size)
                
            samples, labels = X_train[idx], y_train[idx]
            samples, labels = shuffle(samples, labels)
            
            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_samples = self.generator.predict([noise, labels])

            # label smoothing
            if epoch < epochs//1.5:
                valid_smooth = (valid+0.1)-(np.random.random(valid.shape)*0.1)
                fake_smooth = (fake-0.1)+(np.random.random(fake.shape)*0.1)
            else:
                valid_smooth = valid
                fake_smooth = fake
                
            # Train the discriminator
            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch([samples, labels], valid_smooth)
            d_loss_fake = self.discriminator.train_on_batch([gen_samples, labels], fake_smooth)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            self.discriminator.trainable = False
            sampled_labels = np.random.randint(0, 2, batch_size).reshape(-1, 1)
            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            if (epoch+1)%sample_interval==0:
                print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
                  % (epoch, epochs, d_loss[0], g_loss[0]))
            G_losses.append(g_loss[0])
            D_losses.append(d_loss[0])
            if plot:
                if epoch+1==epochs:
                    plt.figure(figsize=(10,5))
                    plt.title("Generator and Discriminator Loss")
                    plt.plot(G_losses,label="G")
                    plt.plot(D_losses,label="D")
                    plt.xlabel("iterations")
                    plt.ylabel("Loss")
                    plt.legend()
                    plt.show()


In [4]:
df = pd.read_csv('./GAN1.csv')
df.head()

,Index,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,R2,R3,R4,R5,R6,R7,R8,t,L,max_deflection (mm)
0,6137,14.7,8.8,20.3,6.2,18.4,33.0,31.0,3.20,-8.02,...,6.6,8.0,6.4,7.8,6.6,7.6,7.2,1.5,970,103.348335
1,970,5.7,0.3,1.0,3.1,7.2,33.5,32.0,3.06,4.38,...,6.4,6.2,7.2,7.0,6.8,7.0,6.0,1.6,924,110.321869
2,699,4.3,6.0,20.9,7.2,2.9,27.0,28.5,0.24,1.08,...,6.4,6.0,7.6,7.8,6.6,8.0,7.6,1.2,1024,164.886597
3,16717,11.6,9.2,28.1,10.7,7.9,31.0,26.5,-3.30,-5.07,...,7.6,6.8,6.4,6.0,8.0,7.0,6.6,1.7,952,121.930344
4,1344,19.4,7.6,20.0,8.9,19.5,30.0,27.5,3.18,-7.90,...,7.4,7.4,7.6,8.0,7.4,8.0,7.8,1.6,1036,135.297989


In [5]:
# le = preprocessing.LabelEncoder()
# for i in ['workclass','education','marital.status','occupation','relationship','race','sex','native.country','income']:
#     df[i] = le.fit_transform(df[i].astype(str))

In [6]:
df.head()

,Index,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,R2,R3,R4,R5,R6,R7,R8,t,L,max_deflection (mm)
0,6137,14.7,8.8,20.3,6.2,18.4,33.0,31.0,3.20,-8.02,...,6.6,8.0,6.4,7.8,6.6,7.6,7.2,1.5,970,103.348335
1,970,5.7,0.3,1.0,3.1,7.2,33.5,32.0,3.06,4.38,...,6.4,6.2,7.2,7.0,6.8,7.0,6.0,1.6,924,110.321869
2,699,4.3,6.0,20.9,7.2,2.9,27.0,28.5,0.24,1.08,...,6.4,6.0,7.6,7.8,6.6,8.0,7.6,1.2,1024,164.886597
3,16717,11.6,9.2,28.1,10.7,7.9,31.0,26.5,-3.30,-5.07,...,7.6,6.8,6.4,6.0,8.0,7.0,6.6,1.7,952,121.930344
4,1344,19.4,7.6,20.0,8.9,19.5,30.0,27.5,3.18,-7.90,...,7.4,7.4,7.6,8.0,7.4,8.0,7.8,1.6,1036,135.297989


In [6]:
# df.income.value_counts()

income
0    24720
1     7841
Name: count, dtype: int64

In [7]:
scaler = StandardScaler()

X = scaler.fit_transform(df.drop('max_deflection (mm)', axis=1))
y = df['max_deflection (mm)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
# lgb_1 = lgb.LGBMRegressor()
# lgb_1.fit(X_train, y_train)

# y_pred = lgb_1.predict(X_test)

# # evaluation
# print(classification_report(y_test, y_pred))

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001170 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4029
[LightGBM] [Info] Number of data points in the train set: 11131, number of used features: 28
[LightGBM] [Info] Start training from score 128.213500


In [14]:
cgan = cGAN()

C:\Users\pokar\AppData\Roaming\Python\Python311\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [15]:
y_train = y_train.reshape(-1,1)
pos_index = np.where(y_train==1)[0]
neg_index = np.where(y_train==0)[0]
cgan.train(X_train, y_train, pos_index, neg_index, epochs=500)

InvalidArgumentError: Graph execution error:

Detected at node 'Generator/embedding_1/embedding_lookup' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\pokar\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\pokar\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\pokar\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\pokar\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Python311\Lib\asyncio\base_events

In [27]:
# we want to generate 19758 instances with class value 0 since that represents how many 0s are in the label of the real training set
noise = np.random.normal(0, 1, (19758, 32))
sampled_labels = np.zeros(19758).reshape(-1, 1)


gen_samples = cgan.generator.predict([noise, sampled_labels])

gen_df = pd.DataFrame(data = gen_samples,
                      columns = df.drop('income',axis=1).columns)

618/618 [==============================] - 1s 1ms/step


In [29]:
# we want to generate 6290 instances with class value 1 since that represents how many 1s are in the label of the real training set
noise_2 = np.random.normal(0, 1, (6290, 32))
sampled_labels_2 = np.ones(6290).reshape(-1, 1)


gen_samples_2 = cgan.generator.predict([noise_2, sampled_labels_2])

gen_df_2 = pd.DataFrame(data = gen_samples_2,
                      columns = df.drop('income',axis=1).columns)

197/197 [==============================] - 0s 1ms/step


In [31]:
gen_df_2['income'] = 1
gen_df['income']=0

df_gan = pd.concat([gen_df_2, gen_df], ignore_index=True, sort=False)
df_gan = df_gan.sample(frac=1).reset_index(drop=True)

X_train_2 = df_gan.drop('income', axis=1)
y_train_2 = df_gan['income'].values

In [32]:
df_gan.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,-0.922658,0.662511,-0.894895,-0.195383,0.999999,0.995299,0.943640,0.282339,0.999829,-0.938371,0.073943,-0.955486,0.986462,-0.994508,0
1,0.939268,-0.898944,0.930043,0.999967,-0.858940,-0.982082,0.448565,0.339969,0.999987,-0.002682,-0.863964,-0.999209,-0.919217,0.999918,0
2,-0.661658,0.776845,-0.892718,-0.996774,0.995564,0.333591,-0.474191,0.940190,-0.541956,-0.928204,0.853827,0.580300,-0.155477,-0.291948,0
3,0.561095,0.366784,-0.978321,-0.848899,0.999999,0.242498,0.999629,-0.702950,0.840505,-0.998661,0.789120,0.948829,0.968598,0.025332,1
4,0.968308,-0.982492,0.994344,0.990058,-0.999998,-0.998776,-0.524285,0.964401,0.968566,-0.980577,0.968241,0.999984,-0.995492,0.998736,0
